In [42]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import streamlit as st
import geopy.distance
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
import osmnx 
import numpy as np
import geocoder
import datapackage
import geopip
import ast
from haversine import haversine, Unit
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# The Big Query Credentials are in the path .streamlit/secrets.toml 
# First we save this credentials in the variable credentials and then we create the connection with bigquery
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
BigQuery_client = bigquery.Client(credentials=credentials)
geolocator = Nominatim(user_agent="geoapiExercises")
# The ofertas query acces the locations of the accounts and the location of the rep when the visit was registered.
datasets = list(BigQuery_client.list_datasets())  # Make an API request.
project = BigQuery_client.project

In [2]:
def distance_rows(start, points):
        return min([haversine(start,stop,unit=Unit.FEET) for stop in points])

def get_country(Latitude,Longitude):
    location = geolocator.reverse(str(Latitude)+","+str(Longitude))
    address = location.raw['address']
    return address['country']

In [45]:
cuentas_q="""
SELECT a.id,a.practice_name,p.longitude as acc_lon,p.latitude as acc_lat,p.state,p.city, p.country
FROM `star-big-data.star_us_rds.accounts` a
LEFT JOIN `star-big-data.star_us_rds.places` p ON a.place_id=p.id
"""
ofertas_q="""
SELECT o.id,CAST(o.offer_date as DATE) as date,o.offer_date,o.account_id,a.practice_name,r.name,o.longitude as visit_lon,o.latitude as visit_lat,p.longitude as acc_lon,p.latitude as acc_lat,p.state,p.city,o.status,o.total_offer
FROM `star-big-data.star_us_rds.offers` o
LEFT JOIN `star-big-data.star_us_rds.accounts` a ON o.account_id=a.id
LEFT JOIN `star-big-data.star_us_rds.places` p ON a.place_id=p.id
LEFT JOIN `star-big-data.star_us_rds.reps` r ON o.rep_id = r.id
WHERE r.name <> 'General Pot' and status <> 'NoVisit' and account_id IS NOT NULL and (p.longitude IS NOT NULL and p.latitude IS NOT NULL) and (o.longitude IS NOT NULL and o.latitude IS NOT NULL) and r.inactive=False
ORDER BY offer_date desc
"""
tracks="""
    SELECT date,name, STRING_AGG(coordinates) as coords
    FROM(
        SELECT CAST(timestamp as DATE) as date, r.name, CONCAT("(",latitude,",",longitude,")") as coordinates
        FROM `star-big-data.star_us_rds.Track` t
        LEFT JOIN `star-big-data.star_us_rds.reps` r ON t.rep_id = r.id 
    )
    GROUP BY date,name
"""

#cuentas=(BigQuery_client.query(cuentas_q).result().to_dataframe(create_bqstorage_client=True,))
#cuentas = cuentas.dropna(subset=['acc_lon','acc_lat'])
#cuentas['country_info']=cuentas.apply(lambda row : geopip.search(lng=row["acc_lon"],lat=row["acc_lat"])['NAME'] if geopip.search(lng=row["acc_lon"],lat=row["acc_lat"]) else '', axis=1)

coordinates=(BigQuery_client.query(tracks).result().to_dataframe(create_bqstorage_client=True,))
coordinates = coordinates.dropna()
coordinates['coords']=coordinates['coords'].apply(lambda x: list(ast.literal_eval(x)))

ofertas = (BigQuery_client.query(ofertas_q).result().to_dataframe(create_bqstorage_client=True,))
ofertas =ofertas.loc[:,~ofertas.columns.str.startswith('_')]
ofertas['acc_coords']=ofertas[['acc_lat','acc_lon']].apply(lambda x: (x[0],x[1]),axis=1)
ofertas['visit_coords']=ofertas[['visit_lat','visit_lon']].apply(lambda x: (x[0],x[1]),axis=1)


consolidado=ofertas.merge(coordinates,how='left',on=['date','name']).dropna()
consolidado['acc_route_dist']=(consolidado[['acc_coords','coords']].apply(lambda x : distance_rows(x[0],x[1]),axis=1))

In [46]:
consolidado['acc_route_dist'].describe().round()

count      44710.000
mean        5530.000
std       175878.000
min            0.000
25%          140.000
50%          354.000
75%          724.000
max     13003384.000
Name: acc_route_dist, dtype: float64

In [54]:
np.quantile(consolidado['acc_route_dist'],0.87)

1065.4488717477482

In [48]:
consolidado[(consolidado['acc_route_dist']>10000)&(consolidado['acc_route_dist']<13003385)].sort_values('acc_route_dist',ascending=False).head()

,id,date,offer_date,account_id,practice_name,name,visit_lon,visit_lat,acc_lon,acc_lat,state,city,status,total_offer,acc_coords,visit_coords,coords,acc_route_dist
44397,1443059,2022-01-04,2022-01-04 19:19:50+00:00,50129,Chang Vicky DDS,Eran Even-Kesef,-122.080,37.631,-75.253,39.980,PA,PHILADELPHIA,NoOffer,0.000,"(39.979604, -75.253272)","(37.63133696754376, -122.07984950205616)","[(37.701433679443724, -121.84895576110036), (3...",13003384.196
42985,1445428,2022-01-12,2022-01-12 17:43:47+00:00,49039,Reish Bruce D DDS,Eran Even-Kesef,-122.466,37.689,-75.255,41.005,PA,Stroudsburg,NoOffer,0.000,"(41.00467, -75.25503)","(37.68934705298179, -122.46620767761252)","[(37.797389758352516, -122.41015536541153), (3...",12931954.666
43226,1445047,2022-01-11,2022-01-11 19:35:23+00:00,535601,JUNKIN DENTAL ASSOC PC,Elizabeth Reynoso,-121.498,38.522,-75.114,40.137,PA,Abington,NoOffer,0.000,"(40.13683, -75.11359)","(38.5223388671875, -121.4975771774738)","[(38.594557782137024, -121.38293729411866), (3...",12895048.904
42821,1445693,2022-01-12,2022-01-12 22:33:35+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.754846555216005, -122.47661557059965)","[(37.797389758352516, -122.41015536541153), (3...",12463329.877
42641,1445964,2022-01-13,2022-01-13 18:34:05+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.75490874893754, -122.47651037771485)","[(37.524220049777156, -122.25269875328723), (3...",12462271.071


In [49]:
consolidado.loc[(consolidado['acc_route_dist']>10000)&(consolidado['acc_route_dist']<13003385),'name'].value_counts()

Jim Harness            110
Brent Hicks             95
Sammie Harris           88
Patrick Markowski       35
Jason Mankevich         33
Fernando Andrews        32
Judd Stewart            29
Brian Sundberg          29
Whitney Ross            27
Shay Bar Nissim         25
Vicky Barnett           25
Eran Even-Kesef         24
Elizabeth Reynoso       24
Robert Gonshorowski     23
Alex Aguilar            21
Debra Morris            20
Demery Moody            19
Robert Gregg            18
Eric Moskow             18
Michael Beller          18
Jordan Hunter           18
Joseph Bonham           16
Chris Charbonneau       13
Aaron Domke             12
Troy C-Rep              10
David Gulakowski         7
Paul Kanengiser          3
Loren Feldman            2
Name: name, dtype: int64

In [119]:
index = 43224
latlon = list(consolidado['coords'][index])
mapit = folium.Map( location=latlon[0], zoom_start=6 ,tiles='Stamen Toner')
for coord in latlon:
    folium.Circle( location=[ coord[0], coord[1] ], color='#43d9de', radius=10 ).add_to( mapit )
folium.Circle(location=[consolidado['visit_coords'][index][0],consolidado['visit_coords'][index][1]],color='red',radius=10).add_to( mapit )
folium.Circle(location=[consolidado['acc_coords'][index][0],consolidado['acc_coords'][index][1]],color='yellow',radius=10).add_to( mapit )
mapit.fit_bounds(mapit.get_bounds())
mapit